In [145]:
import numpy as np
import math
import matplotlib.pyplot
from matplotlib import pyplot as plt
from keras.utils import np_utils
from keras.datasets import mnist

In [146]:
maxAcc = 0.0
maxIter = 0
C_Lambda = 0.02
TrainingPercent = 80
ValidationPercent = 10
TestPercent = 10
M = 10 #The number of basis functions
PHI = []
IsSynthetic = False

# Function to generate target values which will be used as a reference for relevance testing
def GetTargetVector(filePath):
    t = []
    with open(filePath, 'rU') as f:
        reader = csv.reader(f)
        for row in reader:  
            t.append(int(row[0]))
    #print("Raw Training Generated..")
    #print (t)
    return t

# def GenerateRawData(filePath, IsSynthetic):    
#     dataMatrix = [] 
#     with open(filePath, 'rU') as fi:
#         reader = csv.reader(fi)
#         for row in reader:
#             dataRow = []
#             for column in row:
#                 dataRow.append(float(column)+0.0001)
#             dataMatrix.append(dataRow)   
    
# #     if IsSynthetic == False :
        
# #         Since columns 5 to 9 have 0-numbered values we remove them from the matrix.
# #         Keeping these columns would have made the matrix un-invertible.
# #          dataMatrix = np.delete(dataMatrix, [5,6,7,8,9], axis=1)
#     dataMatrix = np.transpose(dataMatrix)     
#     #print ("Data Matrix Generated..")
#     return dataMatrix

def GenerateRawData(filePath, IsSynthetic):    
    dataMatrix = [] 
    with open(filePath, 'rU') as fi:
        reader = csv.reader(fi)
        for row in reader:
            dataRow = []
            for column in row:
                dataRow.append(float(column)+0.0001)
            dataMatrix.append(dataRow)   
    
#     if IsSynthetic == False :
        
#         Since columns 5 to 9 have 0-numbered values we remove them from the matrix.
#         Keeping these columns would have made the matrix un-invertible.
#          dataMatrix = np.delete(dataMatrix, [5,6,7,8,9], axis=1)
    dataMatrix = np.transpose(dataMatrix)     
    #print ("Data Matrix Generated..")
    return dataMatrix




# This function is used to generate training vector
def GenerateTrainingTarget(rawTraining,TrainingPercent = 80):
    TrainingLen = int(math.ceil(len(rawTraining)*(TrainingPercent*0.01)))
    t           = rawTraining[:TrainingLen]
    #print(str(TrainingPercent) + "% Training Target Generated..")
    return t
 
def GenerateTrainingDataMatrix(rawData, TrainingPercent = 80):
    T_len = int(math.ceil(len(rawData[0])*0.01*TrainingPercent))
    d2 = rawData[:,0:T_len]
    #print(T_len)
    #print(str(TrainingPercent) + "% Training Data Generated..")
    return d2

def GenerateValData(rawData, ValPercent, TrainingCount): 
    valSize = int(math.ceil(len(rawData[0])*ValPercent*0.01))
    V_End = TrainingCount + valSize
    dataMatrix = rawData[:,TrainingCount+1:V_End]
    #print (str(ValPercent) + "% Val Data Generated..")  
    return dataMatrix

def GenerateValTargetVector(rawData, ValPercent, TrainingCount): 
    valSize = int(math.ceil(len(rawData)*ValPercent*0.01))
    V_End = TrainingCount + valSize
    t =rawData[TrainingCount+1:V_End]
    #print (str(ValPercent) + "% Val Target Data Generated..")
    return t

#
def GenerateBigSigma(Data, MuMatrix,TrainingPercent,IsSynthetic):
    BigSigma    = np.zeros((len(Data),len(Data)))
    DataT       = np.transpose(Data)
    TrainingLen = math.ceil(len(DataT)*(TrainingPercent*0.01))        
    varVect     = []
#     print(BigSigma)
#     print(DataT)
#     print(TrainingLen)
    for i in range(0,len(DataT[0])):
        vct = []
        for j in range(0,int(TrainingLen)):
            vct.append(Data[i][j])    
        varVect.append(np.var(vct))
    
    for j in range(len(Data)):
        BigSigma[j][j] = varVect[j]
    if IsSynthetic == True:
        BigSigma = np.dot(3,BigSigma)
    else:
        BigSigma = np.dot(200,BigSigma)
    ##print ("BigSigma Generated..")
    return BigSigma

def GetScalar(DataRow,MuRow, BigSigInv):  
    R = np.subtract(DataRow,MuRow)
    T = np.dot(BigSigInv,np.transpose(R))  
    L = np.dot(R,T)
    return L
# This function geberates 
def GetRadialBasisOut(DataRow,MuRow, BigSigInv):    
    phi_x = math.exp(-0.5*GetScalar(DataRow,MuRow,BigSigInv))
    return phi_x

def GetPhiMatrix(Data, MuMatrix, BigSigma, TrainingPercent = 80):  
    # Generates a phi design matrix. The vector data is converted into a scalar.
    DataT = np.transpose(Data)
    TrainingLen = math.ceil(len(DataT)*(TrainingPercent*0.01))         
    PHI = np.zeros((int(TrainingLen),len(MuMatrix))) 
    BigSigInv = np.linalg.inv(BigSigma)
    for  C in range(0,len(MuMatrix)):
        for R in range(0,int(TrainingLen)):
            PHI[R][C] = GetRadialBasisOut(DataT[R], MuMatrix[C], BigSigInv)
    #print ("PHI Generated..")
    return PHI

# Function generates training weights for closed form solution
def GetWeightsClosedForm(PHI, T, Lambda):
    Lambda_I = np.identity(len(PHI[0]))
    for i in range(0,len(PHI[0])):
        Lambda_I[i][i] = Lambda
    PHI_T       = np.transpose(PHI)
    PHI_SQR     = np.dot(PHI_T,PHI)
    PHI_SQR_LI  = np.add(Lambda_I,PHI_SQR)
    PHI_SQR_INV = np.linalg.inv(PHI_SQR_LI)
    INTER       = np.dot(PHI_SQR_INV, PHI_T)
    W           = np.dot(INTER, T)
    ##print ("Training Weights Generated..")
    return W

# def GetPhiMatrix(Data, MuMatrix, BigSigma, TrainingPercent = 80):
#     DataT = np.transpose(Data)
#     TrainingLen = math.ceil(len(DataT)*(TrainingPercent*0.01))         
#     PHI = np.zeros((int(TrainingLen),len(MuMatrix))) 
#     BigSigInv = np.linalg.inv(BigSigma)
#     for  C in range(0,len(MuMatrix)):
#         for R in range(0,int(TrainingLen)):
#             PHI[R][C] = GetRadialBasisOut(DataT[R], MuMatrix[C], BigSigInv)
#     #print ("PHI Generated..")
#     return PHI

def GetSigmoidValTest(VAL_PHI,W):
#     sigmoid =  (1/(1+np.e**(-np.dot(TRAINING_PHI[i],np.transpose(W_Now)))))
#     Y = np.dot(W,np.transpose(VAL_PHI))
    Y = (1/(1+np.e**(-np.dot(VAL_PHI,np.transpose(W)))))
    ##print ("Test Out Generated..")
#     print(Y)
    return Y

def GetSoftMaxValTest(VAL_FEATURE,W):
#     sigmoid =  (1/(1+np.e**(-np.dot(TRAINING_PHI[i],np.transpose(W_Now)))))
#     Y = np.dot(W,np.transpose(VAL_PHI))
#     Y = (1/(1+np.e**(-np.dot(VAL_FEATURE,np.transpose(W)))))
    z = (np.dot(VAL_FEATURE,np.transpose(W)))
    
    Y = (np.exp(z)/np.sum(np.exp(z)))
    
    ##print ("Test Out Generated..")
#     print(Y)
    return Y

def GetValTest(VAL_PHI,W):
    Y = np.dot(W,np.transpose(VAL_PHI))
    ##print ("Test Out Generated..")
    return Y

# This function is used to find RMSE
def GetErms(VAL_TEST_OUT,ValDataAct):
    sum = 0.0
    t=0
    accuracy = 0.0
    counter = 0
    val = 0.0
    
    print(ValDataAct.shape)
    print(VAL_TEST_OUT.shape)
    
    for i in range (0,len(VAL_TEST_OUT)):
        
        print(ValDataAct[i])
        print(VAL_TEST_OUT[i])
        # We generate the sum of squared error. This sum is further used to calculate the root mean squared error.
        sum = sum + math.pow((ValDataAct[i] - VAL_TEST_OUT[i]),2)
        
        # The counter is incremented every time predicted and target values are same. 
        # This counter value is used to calculate the accuracy
        if(int(np.around(VAL_TEST_OUT[i], 0)) == ValDataAct[i]):
#         if(int(VAL_TEST_OUT[i]) >= 0.1):
            counter+=1
    accuracy = (float((counter*100))/float(len(VAL_TEST_OUT)))
    ##print ("Accuracy Generated..")
    ##print ("Validation E_RMS : " + str(math.sqrt(sum/len(VAL_TEST_OUT))))
    return (str(accuracy) + ',' +  str(math.sqrt(sum/len(VAL_TEST_OUT))))

def GetCCE(VAL_TEST_OUT,ValDataAct):
    sum = 0.0
    t=0
    accuracy = 0.0
    counter = 0
    val = 0.0
    
    for i in range (0,len(VAL_TEST_OUT)):
        # We generate the sum of squared error. This sum is further used to calculate the root mean squared error.
        
        op = ValDataAct[i][np.argmax(ValDataAct[i])]
        predict = VAL_TEST_OUT[i][np.argmax(ValDataAct[i])]
        
        sum = sum-op*math.log(predict)
        
        # The counter is incremented every time predicted and target values are same. 
        # This counter value is used to calculate the accuracy
        if(np.argmax(ValDataAct[i]) == np.argmax(VAL_TEST_OUT[i])):
#         if(int(predict) == op):
#         if(int(VAL_TEST_OUT[i]) >= 0.1):
            counter = counter + 1
    accuracy = (float((counter*100))/float(len(VAL_TEST_OUT)))
    ##print ("Accuracy Generated..")
    ##print ("Validation E_RMS : " + str(math.sqrt(sum/len(VAL_TEST_OUT))))
    return (str(accuracy) + ',' +  str(math.sqrt(sum/len(VAL_TEST_OUT))))


def softMax(z):
    s_max = (np.exp(z)/np.sum(np.exp(z)))
    return s_max

def oneHotIt(Y):
    m = Y.shape[0]
    #Y = Y[:,0]
    OHX = scipy.sparse.csr_matrix((np.ones(m), (Y, np.array(range(m)))))
    OHX = np.array(OHX.todense()).T
    return OHX

def getAccuracy(someX,someY):
    prob,prede = getProbsAndPreds(someX)
    accuracy = sum(prede == someY)/(float(len(someY)))
    return accuracy

def getProbsAndPreds(someX):
    probs = softMax(np.dot(someX,w))
    preds = np.argmax(probs,axis=1)
    return probs,preds


def softMax1(y_linear):
    exp = np.exp(y_linear-np.max(y_linear).reshape((-1,1)))
    norms = np.sum(exp).reshape((-1,1))
    return exp / norms
    

    
    

In [147]:

trainLen = 60000
testLen = 10000
imageVector = 784
numClasses = 10

(trainData, trainLabel), (testData, testLabel) = mnist.load_data()

trainData = trainData.reshape(trainLen,imageVector)
testData = testData.reshape(testLen,imageVector)

trainData = trainData.astype("float32")
testData = testData.astype("float32")

trainData = trainData/255
testData = testData/255

# exampleNumber = 2 #Pick the example we want to visualize
# example = trainData[0][exampleNumber,:] #Then we load that example.
# plt.imshow(np.reshape(example,[28,28])) #Next we reshape it to 28x28 and display it.



TrainingTarget = np_utils.to_categorical(trainLabel,numClasses)
TrainingData   = (trainData)
TestDataAct = np_utils.to_categorical(testLabel,numClasses)
TestData = (testData)

print(TrainingData.shape)
print(TrainingTarget.shape)
print(TestData.shape)
print(TestDataAct.shape)


Wts          = np.random.normal(loc = 0.0, scale = 0.01, size = (len(TrainingTarget[0]),len(TrainingData[0])))


# Wts = np.zeros([TrainingData.shape[1],len(np.unique(numClasses))])


W_Now        = np.dot(1, Wts)
bias = np.zeros((1,1))
print(W_Now.shape)
La           = 0.0001
learningRate = 0.1
L_Erms_Val   = []
L_Erms_TR    = []
L_Erms_Test  = []
W_Mat        = []
Accuracy_Test = []
logLosses = []

    
batchSize = 10000

for i in range(0,2):    
#     print(TrainingData[i].shape)
#     z = (np.dot(np.transpose(W_Now),TrainingData[i]))
    print(i)
    for j in range(0,trainLen,batchSize):
        sumDelta_ED = 0
        for k in range(j, j+batchSize):
            z = (np.dot((W_Now),TrainingData[k]))
            soft_max = softMax(z)
            error = (TrainingTarget[k] - soft_max).reshape(10,1)
            train = TrainingData[k].reshape(1,784)
            Delta_ED     = -np.dot(error,train)
            sumDelta_ED += Delta_ED
    
        Delta_E_D = (sumDelta_ED/batchSize)
        La_Delta_E_W  = np.dot(La,W_Now)
        Delta_E       = np.add(Delta_E_D,La_Delta_E_W)    
        Delta_W       = -np.dot(learningRate,Delta_E)
        W_T_Next      = W_Now + Delta_W
        W_Now         = W_T_Next

        #-----------------TrainingData Accuracy---------------------#
    #     TR_TEST_OUT   = GetValTest(TRAINING_PHI,W_T_Next) 
    #     TR_TEST_OUT = GetSigmoidValTest(TRAINING_PHI,W_T_Next)
        TR_TEST_OUT = GetSoftMaxValTest(TrainingData,W_T_Next)
    #     Erms_TR       = GetErms(TR_TEST_OUT,TrainingTarget)
        Erms_TR       = GetCCE(TR_TEST_OUT,TrainingTarget)

        L_Erms_TR.append(float(Erms_TR.split(',')[1]))

        #-----------------ValidationData Accuracy---------------------#
    #     VAL_TEST_OUT  = GetValTest(VAL_PHI,W_T_Next)
    #     VAL_TEST_OUT = GetSigmoidValTest(VAL_PHI,W_T_Next)
    #     VAL_TEST_OUT = GetSoftMaxValTest(ValData,W_T_Next)
    #     Erms_Val      = GetErms(VAL_TEST_OUT,ValDataAct)
    #     L_Erms_Val.append(float(Erms_Val.split(',')[1]))

        #-----------------TestingData Accuracy---------------------#
    #     TEST_OUT      = GetValTest(TEST_PHI,W_T_Next)
    #     TEST_OUT = GetSigmoidValTest(TEST_PHI,W_T_Next)
        TEST_OUT = GetSoftMaxValTest(TestData,W_T_Next)
        Erms_Test = GetCCE(TEST_OUT,TestDataAct)
        L_Erms_Test.append(float(Erms_Test.split(',')[1]))



    #     logLoss = np.mean(-y * np.log(sigmoid) - (1 - y) * np.log(1 - sigmoid))
    #     print(str(logLoss)) #logLosses.append((logLoss))

        Accuracy_Test.append(float(Erms_Test.split(',')[0]))
    #     print(str(float(Erms_Test.split(',')[0])))
  
    
plt.plot(L_Erms_Test)
plt.show()

plt.plot(Accuracy_Test)
plt.show()



# plt.plot(logLoss)
# plt.show()

print ('----------Logistic Regression Solution--------------------')
print ("M = "+str(M)+" \nLambda  = "+str(La)+" \neta= "+str(learningRate))
print ("E_rms Training   = " + str(np.around(min(L_Erms_TR),5)))
# print ("E_rms Validation = " + str(np.around(min(L_Erms_Val),5)))
print ("E_rms Testing    = " + str(np.around(min(L_Erms_Test),5)))
print ("Training Accuracy = " + (str(float(Erms_TR.split(',')[0]))))
# print ("Validation Accuracy = " + (str(float(Erms_Val.split(',')[0]))))
print ("Testing Accuracy = " + (str(float(Erms_Test.split(',')[0]))))

# print ("Log Loss   = " + str(float(np.around(min(logLoss),5))))

# print(Wts)

(60000, 784)
(60000, 10)
(10000, 784)
(10000, 10)
